In [3]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Set display options for better understanding of the data
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [5]:
# Load the Excel file
df = pd.read_excel('Megamart_Sales_Performance_Dataset.xlsx')
print(f"Original dataset shape: {df.shape}")
print("\nFirst five rows:")
df.head()

Original dataset shape: (300, 15)

First five rows:


,Order ID,Order Date,Customer ID,Customer Name,Region,Product Category,Product Name,Quantity Sold,Unit Price,Total Sales,Profit Margin (%),Profit Amount,Payment Method,Order Status,Customer Segment
0,ORD0001,2024-02-17,CUST372,Customer 46,East,Furniture,Table,1,402.93,402.93,17.48,70.43,PayPal,Canceled,Wholesale
1,ORD0002,2024-02-22,CUST962,Customer 94,North,Electronics,Laptop,5,399.10,1995.50,15.61,311.50,Credit Card,Pending,Corporate
2,ORD0003,2024-01-12,CUST233,Customer 20,East,Electronics,Smartphone,6,155.03,930.18,26.69,248.27,Bank Transfer,Pending,Retail
3,ORD0004,2024-02-25,CUST927,Customer 117,North,Groceries,Eggs,2,195.79,391.58,5.40,21.15,Bank Transfer,Completed,Wholesale
4,ORD0005,2024-01-24,CUST950,Customer 12,East,Groceries,Eggs,1,63.77,63.77,25.53,16.28,Credit Card,Canceled,Corporate


In [6]:
# Checking the data types and missing values
df.info()

print("Missing values:")
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Order ID           300 non-null    object        
 1   Order Date         300 non-null    datetime64[ns]
 2   Customer ID        300 non-null    object        
 3   Customer Name      300 non-null    object        
 4   Region             300 non-null    object        
 5   Product Category   300 non-null    object        
 6   Product Name       300 non-null    object        
 7   Quantity Sold      300 non-null    int64         
 8   Unit Price         300 non-null    float64       
 9   Total Sales        300 non-null    float64       
 10  Profit Margin (%)  300 non-null    float64       
 11  Profit Amount      300 non-null    float64       
 12  Payment Method     300 non-null    object        
 13  Order Status       300 non-null    object        
 14  Customer S

In [7]:
# Check the basic statistics of the data
df.describe()

,Order Date,Quantity Sold,Unit Price,Total Sales,Profit Margin (%),Profit Amount
count,300,300.000000,300.000000,300.000000,300.000000,300.000000
mean,2024-01-30 05:16:48,5.496667,247.402933,1306.030433,17.810633,227.232600
min,2024-01-01 00:00:00,1.000000,13.960000,27.820000,5.010000,2.040000
25%,2024-01-14 18:00:00,3.000000,123.267500,443.582500,11.790000,72.552500
50%,2024-01-30 00:00:00,5.000000,248.285000,979.735000,17.580000,156.585000
75%,2024-02-14 00:00:00,8.000000,374.217500,1822.850000,24.470000,311.757500
max,2024-02-29 00:00:00,10.000000,496.880000,4730.900000,29.940000,1142.220000
std,NaN,2.910595,143.037822,1087.831867,7.250698,218.120907


In [8]:
# Check for duplicates
print(f"Duplicate Analysis:")
exact_duplicates = df.duplicated().sum()
print(f"Exact duplicates: {exact_duplicates}")

Duplicate Analysis:
Exact duplicates: 0


In [9]:
# Check for unique values in categorical columns
print(f"\nUnique Values in Categorical Columns:")
categorical_cols = ['Product Category', 'Customer Segment', 'Order Status', 'Payment Method', 'Region']   
for col in categorical_cols:
    if col in df.columns:
        unique_count = df[col].nunique()
        print(f"- {col}: {unique_count} unique values")
        print(f"  Values: {df[col].unique()}")


Unique Values in Categorical Columns:
- Product Category: 4 unique values
  Values: ['Furniture' 'Electronics' 'Groceries' 'Clothing']
- Customer Segment: 3 unique values
  Values: ['Wholesale' 'Corporate' 'Retail']
- Order Status: 3 unique values
  Values: ['Canceled' 'Pending' 'Completed']
- Payment Method: 4 unique values
  Values: ['PayPal' 'Credit Card' 'Bank Transfer' 'Cash']
- Region: 4 unique values
  Values: ['East' 'North' 'West' 'South']


In [13]:
# Create a copy to avoid modifying original
df_clean = df.copy()
original_shape = df_clean.shape

In [14]:
# Remove duplicates
before_duplicates = len(df_clean)
df_clean = df_clean.drop_duplicates()
after_duplicates = len(df_clean)
duplicates_removed = before_duplicates - after_duplicates
print(f"Removed {duplicates_removed} duplicate rows")

Removed 0 duplicate rows


In [15]:
# Clean text columns (remove extra spaces)
text_columns = ['Product Category', 'Product Name', 'Customer Segment', 'Order Status', 'Payment Method', 'Region']
    
for col in text_columns:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str).str.strip()
        print(f"Cleaned {col}")

Cleaned Product Category
Cleaned Product Name
Cleaned Customer Segment
Cleaned Order Status
Cleaned Payment Method
Cleaned Region


In [17]:
# Fix data type issues

# Convert Order Date to datetime if it's not already
if 'Order Date' in df_clean.columns:
    df_clean['Order Date'] = pd.to_datetime(df_clean['Order Date'], errors='coerce')
    print("Converted Order Date column to datetime")

# Convert numeric columns
numeric_columns = ['Quantity Sold', 'Unit Price', 'Total Sales', 'Profit Margin (%)', 'Profit Amount']
for col in numeric_columns:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
        print(f"Converted {col} column to numeric")

# Add Month Column
df_clean['Month'] = df_clean['Order Date'].dt.month_name()
df_clean['Month'] = df_clean['Month'].str.slice(0, 3)

# Add Year Column
df_clean['Year'] = df_clean['Order Date'].dt.year

# Add Day Column
df_clean['Day'] = df_clean['Order Date'].dt.day


Converted Order Date column to datetime
Converted Quantity Sold column to numeric
Converted Unit Price column to numeric
Converted Total Sales column to numeric
Converted Profit Margin (%) column to numeric
Converted Profit Amount column to numeric


In [18]:
# Fix any calculation inconsistencies

# Fix Total Sales = Quantity Sold * Unit Price
df_clean['Total Sales'] = df_clean['Quantity Sold'] * df_clean['Unit Price']
print("Recalculated Total Sales")

# Fix Profit Amount = Total Sales * (Profit Margin % / 100)
df_clean['Profit Amount'] = df_clean['Total Sales'] * (df_clean['Profit Margin (%)'] / 100)
print("Recalculated Profit Amount")

Recalculated Total Sales
Recalculated Profit Amount


In [19]:
# Remove invalid numerical records

# Count invalid records before removal
invalid_quantity = (df_clean['Quantity Sold'] <= 0).sum()
invalid_price = (df_clean['Unit Price'] <= 0).sum()
invalid_sales = (df_clean['Total Sales'] <= 0).sum()
invalid_profit_margin = ((df_clean['Profit Margin (%)'] < 0) | (df_clean['Profit Margin (%)'] > 100)).sum()
invalid_dates = df_clean['Order Date'].isna().sum()

print(f"Invalid records found:")
print(f"- Quantity: {invalid_quantity}")
print(f"- Unit Price: {invalid_price}")
print(f"- Total Sales: {invalid_sales}")
print(f"- Profit Margin: {invalid_profit_margin}")
print(f"- Missing dates: {invalid_dates}")

# Remove invalid records
before_invalid = len(df_clean)

df_clean = df_clean[
    (df_clean['Quantity Sold'] > 0) &
    (df_clean['Unit Price'] > 0) &
    (df_clean['Total Sales'] > 0) &
    (df_clean['Profit Margin (%)'] >= 0) &
    (df_clean['Profit Margin (%)'] <= 100) &
    (df_clean['Order Date'].notna())
]

after_invalid = len(df_clean)
invalid_removed = before_invalid - after_invalid
print(f"Removed {invalid_removed} invalid records")

Invalid records found:
- Quantity: 0
- Unit Price: 0
- Total Sales: 0
- Profit Margin: 0
- Missing dates: 0
Removed 0 invalid records


In [20]:
# Round numeric columns for consistency
df_clean['Total Sales'] = df_clean['Total Sales'].round(2)
df_clean['Profit Amount'] = df_clean['Profit Amount'].round(2)
df_clean['Profit Margin (%)'] = df_clean['Profit Margin (%)'].round(2)
print("Rounded numeric values to 2 decimal places")

Rounded numeric values to 2 decimal places


In [21]:
#Final statistics
final_shape = df_clean.shape
total_removed = original_shape[0] - final_shape[0]
    
print(f"Cleaning Summary:")
print(f"- Original records: {original_shape[0]}")
print(f"- Final records: {final_shape[0]}")
print(f"- Records removed: {total_removed}")
print(f"- Data retention: {(final_shape[0]/original_shape[0]*100):.1f}%")

Cleaning Summary:
- Original records: 300
- Final records: 300
- Records removed: 0
- Data retention: 100.0%


In [22]:
# Save the cleaned data to a new CSV file
csv_filename = 'megamart_sales_cleaned_data.csv'
df_clean.to_csv(csv_filename, index=False)
print(f"Exported to CSV: {csv_filename}")

Exported to CSV: megamart_sales_cleaned_data.csv
